In [43]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
import numpy as np
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

In [2]:
df= pd.read_csv('data_5_products.csv', index_col='Complaint ID', usecols=['Product','Sub-product', 'Issue', 'Sub-issue', 'Consumer complaint narrative', 'Complaint ID'])

In [9]:
df = df.fillna("")
# df[df['Consumer complaint narrative'].str.len()<1000] we need to drop short texts

In [10]:
df.count()

Product                         36958
Sub-product                     36958
Issue                           36958
Sub-issue                       36958
Consumer complaint narrative    36958
dtype: int64

In [18]:
df['Category'] = df["Product"] + " - " + df['Sub-product'] 
df

,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Category
Complaint ID,,,,,,
1297939,Credit card,,Other,,Received Capital One charge card offer XXXX. A...,Credit card -
1296693,Credit card,,Rewards,,I 'm a longtime member of Charter One Bank/RBS...,Credit card -
1296727,Debt collection,Payday loan,Improper contact or sharing of info,Talked to a third party about my debt,I received a call from a XXXX XXXX from XXXX @...,Debt collection - Payday loan
1296785,Credit reporting,,Incorrect information on credit report,Information is not mine,I had my purse stolen in 2007. They never foun...,Credit reporting -
1295146,Debt collection,Mortgage,Communication tactics,Threatened to take legal action,"Going through a divorce, my ex and I were unab...",Debt collection - Mortgage
1296880,Credit card,,Late fee,,XXXX Card services was bought out by Capital O...,Credit card -
1296890,Credit card,,Billing disputes,,I was reported late by Discover Card to the re...,Credit card -
1299216,Credit card,,Credit line increase/decrease,,XXXX i receive an email from citibank regardin...,Credit card -
1297589,Debt collection,"Other (i.e. phone, health club, etc.)",False statements or representation,Attempted to collect wrong amount,I agreed to a settlement arrangement on a acco...,"Debt collection - Other (i.e. phone, health cl..."


In [44]:
labels = pd.get_dummies(df['Product'])
texts = df['Consumer complaint narrative']


,Bank account or service,Credit card,Credit reporting,Debt collection,Mortgage
Complaint ID,,,,,
1297939,0,1,0,0,0
1296693,0,1,0,0,0
1296727,0,0,0,1,0
1296785,0,0,1,0,0
1295146,0,0,0,1,0


,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Category
Complaint ID,,,,,,
1296727,Debt collection,Payday loan,Improper contact or sharing of info,Talked to a third party about my debt,I received a call from a XXXX XXXX from XXXX @...,Debt collection - Payday loan
1295146,Debt collection,Mortgage,Communication tactics,Threatened to take legal action,"Going through a divorce, my ex and I were unab...",Debt collection - Mortgage
1296880,Credit card,,Late fee,,XXXX Card services was bought out by Capital O...,Credit card -
1297626,Mortgage,Conventional fixed mortgage,Credit decision / Underwriting,,"To whom it may concern, To those that follow t...",Mortgage - Conventional fixed mortgage
1297327,Credit card,,Identity theft / Fraud / Embezzlement,,I have been a XXXX XXXX XXXX for 4.5 years and...,Credit card -
1297371,Debt collection,"Other (i.e. phone, health club, etc.)",False statements or representation,Attempted to collect wrong amount,I received a letter from Penn Credit Collectio...,"Debt collection - Other (i.e. phone, health cl..."
1297783,Debt collection,I do not know,Cont'd attempts collect debt not owed,Debt is not mine,This company continues to report on my credit ...,Debt collection - I do not know
1297784,Debt collection,I do not know,Cont'd attempts collect debt not owed,Debt is not mine,This company continues to report on my credit ...,Debt collection - I do not know
1297850,Credit reporting,,Incorrect information on credit report,Account status,"I received my auto insurance policy renewal, a...",Credit reporting -
